In [1]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen
import re
from bancor3_simulator.protocol import BancorV3
from bancor3_simulator.core.settings import GlobalSettings
from decimal import Decimal
from fractions import Fraction
import pandas as pd

cooldown_time: int = 7
withdrawal_fee: Decimal = Decimal('0.0025')
whitelisted_tokens: list = ['bnt', 'eth', 'wbtc', 'link']
bnt_funding_limit: Decimal = Decimal('40000')
bnt_min_liquidity: Decimal = Decimal('10000')
price_feeds = pd.DataFrame({'INDX':[0],
                            'bnt':[1.0],
                            'tkn':[3.0]
                            })

network_fee: Decimal = Decimal('0.2')
trading_fee: Decimal = Decimal('0.01')

def read_url(url):
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

In [2]:
# <p align="center">
# <img width="100%" src="https://d9hhrg4mnvzow.cloudfront.net/try.bancor.network/5edb37d6-open-graph-bancor3_1000000000000000000028.png" alt="bancor3" />
# </p>
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# # Simulation Example

In [3]:
v3 = BancorV3(active_users=['dayTrader', 'tknProvider', 'bntProvider'],
              whitelisted_tokens=['tkn','bnt'])

v3.list_tokens, v3.list_users

(['tkn', 'bnt'], ['dayTrader', 'tknProvider', 'bntProvider'])

In [4]:
v3.upload_json_simulation(path='../bancor3_simulator/tests/data/scenarios_2.json')
v3.list_tokens, v3.list_users

(['tkn', 'bnt'], ['dayTrader', 'tknProvider', 'bntProvider'])

In [5]:
v3.describe()

,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol WalletState
0,,bnt=0.000000,bnt=0.000000,bnbnt=0.000000,bnt=0.000000,tkn=0.000000,bnbnt=0.000000
1,bnt=0.000000 tkn=0.000000,tkn=0.000000,tkn=0.000000,bntkn=0.000000,,bnt=0.000000,
2,,,,,,,


In [6]:
unix_timestamp = 0
simulation = []
validation = []
indx = 0
operations_count = len(v3.json_data['operations'])

In [7]:
v3.global_state.price_feeds

,indx,bat,bnt,dai,enj,link,mana,matic,mkr,mln,ocean,omg,ren,rsr,usdc,usdt,wbtc,eth
0,0,0.182214,0.372701,1.008796,0.063398,2.20,0.034675,0.015172,492.57,4.02,0.018389,1.176558,0.081889,0.002620,1.002780,1.002638,9956.32,185.65
1,1,0.182214,0.372701,1.008796,0.063398,2.20,0.034675,0.015172,492.57,4.02,0.018389,1.176558,0.081889,0.002620,1.002780,1.002638,9956.32,185.65
2,2,0.182214,0.372701,1.008796,0.063398,2.20,0.034675,0.015172,492.57,4.02,0.018389,1.176558,0.081889,0.002620,1.002780,1.002638,9956.32,185.65
3,3,0.182195,0.385374,1.010015,0.063418,2.20,0.034666,0.015118,493.42,4.02,0.018423,1.176224,0.082005,0.002614,1.002988,1.002065,9972.69,185.78
4,4,0.182195,0.385374,1.010015,0.063418,2.20,0.034666,0.015118,493.42,4.02,0.018423,1.176224,0.082005,0.002614,1.002988,1.002065,9972.69,185.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052506,1052506,0.786363,4.260000,0.999664,1.860000,28.01,0.890839,1.630000,3825.03,93.38,0.971470,6.210000,0.611442,0.048066,1.000561,1.001124,48694.03,3231.03
1052507,1052507,0.786363,4.260000,0.999664,1.860000,28.01,0.890839,1.630000,3825.03,93.38,0.971470,6.210000,0.611442,0.048066,1.000561,1.001124,48694.03,3231.03
1052508,1052508,0.786363,4.260000,0.999664,1.860000,28.01,0.890839,1.630000,3825.03,93.38,0.971470,6.210000,0.611442,0.048066,1.000561,1.001124,48694.03,3231.03
1052509,1052509,0.785455,4.260000,0.999836,1.870000,27.99,0.887993,1.630000,3817.94,93.13,0.974647,6.220000,0.611192,0.048178,0.999534,1.000742,48696.27,3228.71


In [8]:
v3.global_state.bnt_min_liquidity

Decimal('1000000')

In [9]:
from bancor3_simulator.core.constants import QDECIMALS

for transaction in v3.json_data['operations']:

    # setup
    indx += 1

    transaction_type = transaction['type']
    user_name = transaction['userId']
    mined = transaction['mined']
    amt = transaction['amount']
    if type(amt) == dict:
        v3.global_state.price_feeds = pd.DataFrame({'INDX':[i for i in range(operations_count)],
                            'tkn':[Decimal(amt['bntVirtualBalance']) for i in range(operations_count)],
                            'bnt':[Decimal(amt['baseTokenVirtualBalance']) for i in range(operations_count)],
        })
        amt = Decimal(0)
    else:
        amt = Decimal(amt)

    tkn_name = 'bnt' if 'BNT' in transaction_type else 'tkn'
    target_tkn = 'tkn' if tkn_name == 'bnt' else 'bnt'
    if mined:
        v3.global_state.unix_timestamp += 1
        unix_timestamp = v3.global_state.unix_timestamp
    if 'enable' in transaction_type:
        v3.dao_msig_init_pools(['bnt','tkn'])

    # action
    elif 'deposit' in transaction_type:
        v3.stake(tkn_name=tkn_name, tkn_amt=amt, user_name=user_name, unix_timestamp=unix_timestamp)
    elif 'trade' in transaction_type:
        v3.trade(source_token=tkn_name, target_token=target_tkn, swap_amount=amt, user_name=user_name, unix_timestamp=unix_timestamp)
    elif 'withdraw' in transaction_type:
        id_number = v3.begin_cooldown(tkn_name=tkn_name,
                                      withdraw_value=amt,
                                      user_name=user_name,
                                      unix_timestamp=unix_timestamp)
        v3.withdraw(user_name=user_name, id_number=id_number, unix_timestamp=unix_timestamp)

    # logging
    expected = transaction['expected']
    collect = {
          "indx": [indx],
          "action": [transaction_type],
          "amount": [amt],
          "tkn_dayTrader": [Decimal(expected['tknBalances']['dayTrader']).quantize(QDECIMALS)],
          "tkn_tknProvider": [Decimal(expected['tknBalances']['tknProvider']).quantize(QDECIMALS)],
          "tkn_bntProvider": [Decimal(expected['tknBalances']['bntProvider']).quantize(QDECIMALS)],
          "tkn_masterVault": [Decimal(expected['tknBalances']['masterVault']).quantize(QDECIMALS)],
          "tkn_epVault": [Decimal(expected['tknBalances']['epVault']).quantize(QDECIMALS)],
          "bnt_dayTrader": [Decimal(expected['bntBalances']['dayTrader']).quantize(QDECIMALS)],
          "bnt_tknProvider": [Decimal(expected['bntBalances']['tknProvider']).quantize(QDECIMALS)],
          "bnt_bntProvider": [Decimal(expected['bntBalances']['bntProvider']).quantize(QDECIMALS)],
          "bnt_masterVault": [Decimal(expected['bntBalances']['masterVault']).quantize(QDECIMALS)],
          "bntkn_dayTrader": [Decimal(expected['bntknBalances']['dayTrader']).quantize(QDECIMALS)],
          "bntkn_tknProvider": [Decimal(expected['bntknBalances']['tknProvider']).quantize(QDECIMALS)],
          "bntkn_bntProvider": [Decimal(expected['bntknBalances']['bntProvider']).quantize(QDECIMALS)],
          "bnbnt_dayTrader": [Decimal(expected['bnbntBalances']['dayTrader']).quantize(QDECIMALS)],
          "bnbnt_tknProvider": [Decimal(expected['bnbntBalances']['tknProvider']).quantize(QDECIMALS)],
          "bnbnt_bntProvider": [Decimal(expected['bnbntBalances']['bntProvider']).quantize(QDECIMALS)],
          "bnbnt_bntPool": [Decimal(expected['bnbntBalances']['bntPool']).quantize(QDECIMALS)],
          "bntCurrentPoolFunding": [Decimal(expected['bntCurrentPoolFunding']).quantize(QDECIMALS)],
          "tknStakedBalance": [Decimal(expected['tknStakedBalance']).quantize(QDECIMALS)],
          "bntStakedBalance": [Decimal(expected['bntStakedBalance']).quantize(QDECIMALS)],
          "tknTradingLiquidity": [Decimal(expected['tknTradingLiquidity']).quantize(QDECIMALS)],
          "bntTradingLiquidity": [Decimal(expected['bntTradingLiquidity']).quantize(QDECIMALS)],
    }
    validation.append(pd.DataFrame(collect))

    collect = {
          "indx": [indx],
          "action": [transaction_type],
          "amount": [amt],
          "tkn_dayTrader": [v3.global_state.users['dayTrader'].wallet['tkn'].tkn_amt.quantize(QDECIMALS)],
          "tkn_tknProvider": [v3.global_state.users['tknProvider'].wallet['tkn'].tkn_amt.quantize(QDECIMALS)],
          "tkn_bntProvider": [v3.global_state.users['bntProvider'].wallet['tkn'].tkn_amt.quantize(QDECIMALS)],
          "tkn_masterVault": [v3.global_state.pools['tkn'].vault_tkn.quantize(QDECIMALS)],
          "tkn_epVault": [v3.global_state.pools['tkn'].external_protection_vault.quantize(QDECIMALS)],
          "bnt_dayTrader": [v3.global_state.users['dayTrader'].wallet['bnt'].tkn_amt.quantize(QDECIMALS)],
          "bnt_tknProvider": [v3.global_state.users['tknProvider'].wallet['bnt'].tkn_amt.quantize(QDECIMALS)],
          "bnt_bntProvider": [v3.global_state.users['bntProvider'].wallet['bnt'].tkn_amt.quantize(QDECIMALS)],
          "bnt_masterVault": [v3.global_state.vault_bnt.quantize(QDECIMALS)],
          "bntkn_dayTrader": [v3.global_state.users['dayTrader'].wallet['tkn'].bntkn_amt.quantize(QDECIMALS)],
          "bntkn_tknProvider": [v3.global_state.users['tknProvider'].wallet['tkn'].bntkn_amt.quantize(QDECIMALS)],
          "bntkn_bntProvider": [v3.global_state.users['bntProvider'].wallet['tkn'].bntkn_amt.quantize(QDECIMALS)],
          "bnbnt_dayTrader": [v3.global_state.users['dayTrader'].wallet['bnt'].bntkn_amt.quantize(QDECIMALS)],
          "bnbnt_tknProvider": [v3.global_state.users['tknProvider'].wallet['bnt'].bntkn_amt.quantize(QDECIMALS)],
          "bnbnt_bntProvider": [v3.global_state.users['bntProvider'].wallet['bnt'].bntkn_amt.quantize(QDECIMALS)],
          "bnbnt_bntPool": [v3.global_state.protocol_wallet_bnbnt.quantize(QDECIMALS)],
          "bntCurrentPoolFunding": [v3.global_state.pools['tkn'].bnt_funding_amount.quantize(QDECIMALS)],
          "tknStakedBalance": [v3.global_state.pools['tkn'].staked_tkn.quantize(QDECIMALS)],
          "bntStakedBalance": [v3.global_state.staked_bnt.quantize(QDECIMALS)],
          "tknTradingLiquidity": [v3.global_state.pools['tkn'].tkn_trading_liquidity.quantize(QDECIMALS)],
          "bntTradingLiquidity": [v3.global_state.pools['tkn'].bnt_trading_liquidity.quantize(QDECIMALS)],
    }
    simulation.append(pd.DataFrame(collect))
    print(amt)


bntkn_rate 1
bntkn_amt 30000000
30000000
tkn bnt_bootstrap_liquidity 2000000
0
bntkn_rate 1
bntkn_amt 1000000
case1
1000000
100000
trade2  3 0.0025 0.20 2000
a, b, d, e, x, direction 4000000 12000000.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 0.0025 0.20 2000 tkn
updated_bnt_liquidity  3999334.7775370771538076987168805199133477753707715380769871688051991334777537077153807698716880519913347775370771538076987168805199133477753707715380769872
2000
trade1  4 0.0025 0.20 5332
a, b, d, e, x, direction 3999334.7775370771538076987168805199133477753707715380769871688051991334777537077153807698716880519913347775370771538076987168805199133477753707715380769872 12002000.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 0.0025 0.20 5332 bnt
updated_bnt_liquidity  4004664.1079916057822915486907634189

In [10]:
99999896903188884206590735671475908806811660970152656416973787145591563459320063230288378582183812595217492705737700893876959395200525979915469544453662348
99999835520333544585297520923157814470304358207305955826255979192021954357763390768099773982859804755737880646525099911182613288573444358752376665259170862

99999835520333544585297520923157814470304358207305955826255979192021954357763390768099773982859804755737880646525099911182613288573444358752376665259170862

In [11]:
v3.global_state.bnt_min_liquidity, v3.global_state.bnt_funding_limit

(Decimal('1000000'), Decimal('10000000'))

In [12]:
simulation, validation = pd.concat(simulation), pd.concat(validation)

In [13]:
simulation = simulation.drop('indx', axis=1).reset_index().drop('index', axis=1)
simulation

,action,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,depositTKN,30000000,10000.000000000000000000,70000000.000000000000000000,0E-18,30000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,0E-18,0E-18,30000000.000000000000000000,0E-18,0E-18,0E-18
1,enableTrading,0,10000.000000000000000000,70000000.000000000000000000,0E-18,30000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,2000000.000000000000000000,2000000.000000000000000000,30000000.000000000000000000,2000000.000000000000000000,6000000.000000000000000000,2000000.000000000000000000
2,depositTKN,1000000,10000.000000000000000000,69000000.000000000000000000,0E-18,31000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,4000000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
3,depositBNT,100000,10000.000000000000000000,69000000.000000000000000000,0E-18,31000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,9900000.000000000000000000,...,0E-18,0E-18,0E-18,100000.000000000000000000,3900000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
4,tradeTKN,2000,8000.000000000000000000,69000000.000000000000000000,0E-18,31002000.000000000000000000,0E-18,10664.889185135810698217,0E-18,9900000.000000000000000000,...,0E-18,0E-18,0E-18,100000.000000000000000000,3900000.000000000000000000,4000001.333111148141976337,31000000.000000000000000000,4000001.333111148141976337,12002000.000000000000000000,3999334.777537077153807699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,depositTKN,36883,17849.329770407740647656,62102442.000000000000000000,0E-18,37889708.670229592259352344,0E-18,7318.712463663813911981,0E-18,6783426.000000000000000000,...,0E-18,0E-18,0E-18,1735099.055019285427254587,6783422.156707742187958060,10000019.117811980227248122,37897636.494896782772412326,10000019.117811980227248122,30023259.277360642248949420,10002667.784755823310177166
395,depositBNT,3304,17849.329770407740647656,62102442.000000000000000000,0E-18,37889708.670229592259352344,0E-18,7318.712463663813911981,0E-18,6780122.000000000000000000,...,0E-18,0E-18,0E-18,1738403.047432951090417753,6780118.156707742187958060,10000019.117811980227248122,37897636.494896782772412326,10000019.117811980227248122,30023259.277360642248949420,10002667.784755823310177166
396,withdrawBNT,69534,17849.329770407740647656,62102442.000000000000000000,0E-18,37889708.670229592259352344,0E-18,7318.712463663813911981,0E-18,6780122.000000000000000000,...,0E-18,0E-18,0E-18,1668869.207090388069114012,6780118.156707742187958060,10000019.117811980227248122,37897636.494896782772412326,10000019.117811980227248122,30023259.277360642248949420,10002667.784755823310177166
397,tradeBNT,36,17957.114151895984338224,62102442.000000000000000000,0E-18,37889600.885848104015661776,0E-18,7282.712463663813911981,0E-18,6780122.000000000000000000,...,0E-18,0E-18,0E-18,1668869.207090388069114012,6780118.156707742187958060,10000019.117811980227248122,37897636.711005818337813209,10000019.117811980227248122,30023151.492979154005258852,10002703.766755758689417176


In [14]:
simulation.head(50)

,action,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,depositTKN,30000000,10000.000000000000000000,70000000.000000000000000000,0E-18,30000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,0E-18,0E-18,30000000.000000000000000000,0E-18,0E-18,0E-18
1,enableTrading,0,10000.000000000000000000,70000000.000000000000000000,0E-18,30000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,2000000.000000000000000000,2000000.000000000000000000,30000000.000000000000000000,2000000.000000000000000000,6000000.000000000000000000,2000000.000000000000000000
2,depositTKN,1000000,10000.000000000000000000,69000000.000000000000000000,0E-18,31000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,4000000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
3,depositBNT,100000,10000.000000000000000000,69000000.000000000000000000,0E-18,31000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,9900000.000000000000000000,...,0E-18,0E-18,0E-18,100000.000000000000000000,3900000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
4,tradeTKN,2000,8000.000000000000000000,69000000.000000000000000000,0E-18,31002000.000000000000000000,0E-18,10664.889185135810698217,0E-18,9900000.000000000000000000,...,0E-18,0E-18,0E-18,100000.000000000000000000,3900000.000000000000000000,4000001.333111148141976337,31000000.000000000000000000,4000001.333111148141976337,12002000.000000000000000000,3999334.777537077153807699
5,tradeBNT,5332,23940.072142346676806238,69000000.000000000000000000,0E-18,30986059.927857653323193762,0E-18,5332.889185135810698217,0E-18,9900000.000000000000000000,...,0E-18,0E-18,0E-18,100000.000000000000000000,3900000.000000000000000000,4000001.333111148141976337,31000031.960044395682560013,4000001.333111148141976337,11986059.927857653323193762,4004664.107991605782291549
6,depositBNT,43560,23940.072142346676806238,69000000.000000000000000000,0E-18,30986059.927857653323193762,0E-18,5332.889185135810698217,0E-18,9856440.000000000000000000,...,0E-18,0E-18,0E-18,143559.985482424435119335,3856440.000000000000000000,4000001.333111148141976337,31000031.960044395682560013,4000001.333111148141976337,11986059.927857653323193762,4004664.107991605782291549
7,depositTKN,220800,23940.072142346676806238,68779200.000000000000000000,0E-18,31206859.927857653323193762,0E-18,5332.889185135810698217,0E-18,9856440.000000000000000000,...,0E-18,0E-18,0E-18,143559.985482424435119335,7861102.773326458864624872,8004665.441102753924267886,31220831.960044395682560013,8004665.441102753924267886,24029584.618414981383103222,8009328.215983211564583097
8,tradeBNT,213,24577.500133818265542449,68779200.000000000000000000,0E-18,31206222.499866181734457551,0E-18,5119.889185135810698217,0E-18,9856440.000000000000000000,...,0E-18,0E-18,0E-18,143559.985482424435119335,7861102.773326458864624872,8004665.441102753924267886,31220833.238095506402537428,8004665.441102753924267886,24028947.190423509794367011,8009541.109480386385412585
9,tradeTKN,613,23964.500133818265542449,68779200.000000000000000000,0E-18,31206835.499866181734457551,0E-18,5323.703738525473911297,0E-18,9856440.000000000000000000,...,0E-18,0E-18,0E-18,143559.985482424435119335,7861102.773326458864624872,8004665.849753487537627712,31220833.238095506402537428,8004665.849753487537627712,24029560.190423509794367011,8009337.192764313318859549


In [15]:
validation = validation.drop('indx', axis=1).reset_index().drop('index', axis=1)
validation

,action,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,depositTKN,30000000,10000.000000000000000000,70000000.000000000000000000,0E-18,30000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,0E-18,0E-18,30000000.000000000000000000,0E-18,0E-18,0E-18
1,enableTrading,0,10000.000000000000000000,70000000.000000000000000000,0E-18,30000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,2000000.000000000000000000,2000000.000000000000000000,30000000.000000000000000000,2000000.000000000000000000,6000000.000000000000000000,2000000.000000000000000000
2,depositTKN,1000000,10000.000000000000000000,69000000.000000000000000000,0E-18,31000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,10000000.000000000000000000,...,0E-18,0E-18,0E-18,0E-18,4000000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
3,depositBNT,100000,10000.000000000000000000,69000000.000000000000000000,0E-18,31000000.000000000000000000,0E-18,10000.000000000000000000,0E-18,9900000.000000000000000000,...,0E-18,0E-18,0E-18,100000.000000000000000000,3900000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
4,tradeTKN,2000,8000.000000000000000000,69000000.000000000000000000,0E-18,31002000.000000000000000000,0E-18,10664.889185135810698217,0E-18,9900000.000000000000000000,...,0E-18,0E-18,0E-18,100000.000000000000000000,3900000.000000000000000000,4000001.333111148141976337,31000000.000000000000000000,4000001.333111148141976337,12002000.000000000000000000,3999334.777537077153807699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,depositTKN,36883,17843.180340779883503492,64004314.664415812819109221,0E-18,35987842.155243407297387287,0E-18,7320.922988706987401096,0E-18,8261197.623349625732944047,...,0E-18,0E-18,0E-18,1735096.742662854896616032,8262253.649874946739798897,9997373.358752083284130504,35990997.219994855880358585,9997373.358752083284130504,30002722.540170501638457541,10000000.000000000000000000
395,depositBNT,3304,17843.180340779883503492,64004314.664415812819109221,0E-18,35987842.155243407297387287,0E-18,7320.922988706987401096,0E-18,8257893.623349625732944047,...,0E-18,0E-18,0E-18,1738400.735072824069155190,8258949.657464977567259739,9997373.358752083284130504,35990997.219994855880358585,9997373.358752083284130504,30002722.540170501638457541,10000000.000000000000000000
396,withdrawBNT,69534,17843.180340779883503492,64004314.664415812819109221,0E-18,35987842.155243407297387287,0E-18,7320.922988706987401096,0E-18,8327253.947685884787087112,...,0E-18,0E-18,0E-18,1668866.735072824069155190,8328483.657464977567259739,9997373.358752083284130504,35990997.219994855880358585,9997373.358752083284130504,30002722.540170501638457541,10000000.000000000000000000
397,tradeBNT,36,17950.919729559836167046,64004314.664415812819109221,0E-18,35987734.415854627344723733,0E-18,7284.922988706987401096,0E-18,8327253.947685884787087112,...,0E-18,0E-18,0E-18,1668866.735072824069155190,8328483.657464977567259739,9997373.358752083284130504,35990997.436013680501817560,9997373.358752083284130504,30002614.800781721685793987,10000035.981999935362000582


In [16]:
# mark_results.dtype = simulation[list(mark_results.columns)].dtype


In [17]:
result_df = (1 - simulation.drop('action', axis=1).replace(0,Decimal(0.0000000000000000001)) / validation.replace(0,Decimal(0.0000000000000000001))) * 100

In [18]:
result_df['action'] = simulation['action'].values
result_df['amount'] = simulation['amount'].values
result_df = result_df[['action', 'amount', 'tkn_dayTrader', 'tkn_tknProvider', 'tkn_bntProvider',
       'tkn_masterVault', 'tkn_epVault', 'bnt_dayTrader', 'bnt_tknProvider',
       'bnt_bntProvider', 'bnt_masterVault', 'bntkn_dayTrader',
       'bntkn_tknProvider', 'bntkn_bntProvider', 'bnbnt_dayTrader',
       'bnbnt_tknProvider', 'bnbnt_bntProvider', 'bnbnt_bntPool',
       'bntCurrentPoolFunding', 'tknStakedBalance', 'bntStakedBalance',
       'tknTradingLiquidity', 'bntTradingLiquidity']]
result_df


,action,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,depositTKN,30000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,enableTrading,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,depositTKN,1000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,depositBNT,100000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,tradeTKN,2000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,depositTKN,36883,-0.0344637531561728700123130868669120095120571...,2.97147571126667855447104963287414241595398149...,0,-5.2847472954392852827034983202625549623520755...,0,0.03019462227076246550098502996151354051636253...,0,17.8881040101597424065003817785701926819477162...,...,0,0,0,-0.0001332695966555653014918836024714682139787...,17.8986455249965291443416771366246198796277476...,-0.0264645418847015888538894709068269564166039...,-5.2975450034007126744091191785832445151625598...,-0.0264645418847015888538894709068269564166039...,-0.0684495787428759884755615639682029138856437...,-0.0266778475582331017716600
395,depositBNT,3304,-0.0344637531561728700123130868669120095120571...,2.97147571126667855447104963287414241595398149...,0,-5.2847472954392852827034983202625549623520755...,0,0.03019462227076246550098502996151354051636253...,0,17.8952610768822360166862944367843482243809836...,...,0,0,0,-0.0001330165180196149952691759422200306458653...,17.9058059691715275254882946659543100870729381...,-0.0264645418847015888538894709068269564166039...,-5.2975450034007126744091191785832445151625598...,-0.0264645418847015888538894709068269564166039...,-0.0684495787428759884755615639682029138856437...,-0.0266778475582331017716600
396,withdrawBNT,69534,-0.0344637531561728700123130868669120095120571...,2.97147571126667855447104963287414241595398149...,0,-5.2847472954392852827034983202625549623520755...,0,0.03019462227076246550098502996151354051636253...,0,18.5791373411378601072489481352240255850858527...,...,0,0,0,-0.0001481255220712448192797341871606324658083...,18.5912053674909485970509309406542386228855103...,-0.0264645418847015888538894709068269564166039...,-5.2975450034007126744091191785832445151625598...,-0.0264645418847015888538894709068269564166039...,-0.0684495787428759884755615639682029138856437...,-0.0266778475582331017716600
397,tradeBNT,36,-0.0345075485238107124344033077417973852324460...,2.97147571126667855447104963287414241595398149...,0,-5.2847629917919956432087850726305941588700157...,0,0.03034383543381614704138332978760821048679014...,0,18.5791373411378601072489481352240255850858527...,...,0,0,0,-0.0001481255220712448192797341871606324658083...,18.5912053674909485970509309406542386228855103...,-0.0264645418847015888538894709068269564166039...,-5.2975449718553699081191762160121793596955803...,-0.0264645418847015888538894709068269564166039...,-0.0684496745826874846748247994664351719366407...,-0.0266777515663477605730769278681117757751894...


In [19]:
result_df.head(50)

,action,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,depositTKN,30000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,enableTrading,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,depositTKN,1000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,depositBNT,100000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,tradeTKN,2000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,tradeBNT,5332,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-3.2258031259093188453738947519316507160225601...,0,0,2.49708832759388096552417582942530880062612067...
6,depositBNT,43560,0,0,0,0,0,0,0,0,...,0,0,0,6.96572931962595264819269177460560625080881689...,3.76450184891486032305586048305129536378567070...,0,-3.2258031259093188453738947519316507160225601...,0,0,2.49708832759388096552417582942530880062612067...
7,depositTKN,220800,0,0,0,0,0,0,0,0,...,0,0,0,6.96572931962595264819269177460560625080881689...,1.84676068443346464021765293590163535171693688...,1.24927144970375688885297745466156008777604513...,-3.2029895977140322525401679682396090889067366...,1.24927144970375688885297745466156008777604513...,2.91307574024212488782979679068875266852293775...,3.74563249139082144828626374413796320093918101...
8,tradeBNT,213,0,0,0,0,0,0,0,0,...,0,0,0,6.96572931962595264819269177460560625080881689...,1.84676068443346464021765293590163535171693688...,1.24927144970375688885297745466156008777604513...,0,1.24927144970375688885297745466156008777604513...,2.91315301687032635926591297173385688257422191...,4.99404391003805874810012966034754394736629612...
9,tradeTKN,613,0,0,0,0,0,-1.8783915279947071797103355901829480776997012...,0,0,...,0,0,0,6.96572931962595264819269177460560625080881689...,1.84676068443346464021765293590163535171693688...,1.24927138592649198036367491661817726956795367...,0,1.24927138592649198036367491661817726956795367...,2.91307870161922770664336079505288241081963159...,6.24271382220869927383340178554158515023910742...


In [20]:
result_df.head(20)

,action,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,depositTKN,30000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,enableTrading,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,depositTKN,1000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,depositBNT,100000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,tradeTKN,2000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,tradeBNT,5332,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-3.2258031259093188453738947519316507160225601...,0,0,2.49708832759388096552417582942530880062612067...
6,depositBNT,43560,0,0,0,0,0,0,0,0,...,0,0,0,6.96572931962595264819269177460560625080881689...,3.76450184891486032305586048305129536378567070...,0,-3.2258031259093188453738947519316507160225601...,0,0,2.49708832759388096552417582942530880062612067...
7,depositTKN,220800,0,0,0,0,0,0,0,0,...,0,0,0,6.96572931962595264819269177460560625080881689...,1.84676068443346464021765293590163535171693688...,1.24927144970375688885297745466156008777604513...,-3.2029895977140322525401679682396090889067366...,1.24927144970375688885297745466156008777604513...,2.91307574024212488782979679068875266852293775...,3.74563249139082144828626374413796320093918101...
8,tradeBNT,213,0,0,0,0,0,0,0,0,...,0,0,0,6.96572931962595264819269177460560625080881689...,1.84676068443346464021765293590163535171693688...,1.24927144970375688885297745466156008777604513...,0,1.24927144970375688885297745466156008777604513...,2.91315301687032635926591297173385688257422191...,4.99404391003805874810012966034754394736629612...
9,tradeTKN,613,0,0,0,0,0,-1.8783915279947071797103355901829480776997012...,0,0,...,0,0,0,6.96572931962595264819269177460560625080881689...,1.84676068443346464021765293590163535171693688...,1.24927138592649198036367491661817726956795367...,0,1.24927138592649198036367491661817726956795367...,2.91307870161922770664336079505288241081963159...,6.24271382220869927383340178554158515023910742...


In [21]:
# bnbnt_bntProvider
# mark_results = pd.read_csv('mark_results.csv')
# mark_results

In [22]:
result_df.dtypes

action                   object
amount                   object
tkn_dayTrader            object
tkn_tknProvider          object
tkn_bntProvider          object
tkn_masterVault          object
tkn_epVault              object
bnt_dayTrader            object
bnt_tknProvider          object
bnt_bntProvider          object
bnt_masterVault          object
bntkn_dayTrader          object
bntkn_tknProvider        object
bntkn_bntProvider        object
bnbnt_dayTrader          object
bnbnt_tknProvider        object
bnbnt_bntProvider        object
bnbnt_bntPool            object
bntCurrentPoolFunding    object
tknStakedBalance         object
bntStakedBalance         object
tknTradingLiquidity      object
bntTradingLiquidity      object
dtype: object

In [23]:
mark_results = pd.read_csv('mark_results.csv')
for col in mark_results.columns:
    mark_results[col] = [Decimal(str(val)) for val in mark_results[col].values]



In [31]:
result_df = simulation[list(mark_results.columns)]

In [32]:
overall = result_df / mark_results.replace(0,Decimal(0.0000000000000000001))
overall['action'] = simulation['action'].values
overall['amount'] = simulation['amount'].values


In [33]:
overall.head(50)

,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,bnt_masterVault,...,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity,action
0,30000000,1.00000000000000000,1.00000000000000000,0E+1,1.00000000000000000,0E+1,1.00000000000000000,0E+1,1.00000000000000000,0E+1,...,0E+1,0E+1,0E+1,0E+1,0E+1,1.00000000000000000,0E+1,0E+1,0E+1,depositTKN
1,0,1.00000000000000000,1.00000000000000000,0E+1,1.00000000000000000,0E+1,1.00000000000000000,0E+1,1.00000000000000000,1.00000000000000000,...,0E+1,0E+1,0E+1,1.00000000000000000,1.00000000000000000,1.00000000000000000,1.00000000000000000,1.00000000000000000,1.00000000000000000,enableTrading
2,1000000,1.00000000000000000,1.00000000000000000,0E+1,1.00000000000000000,0E+1,1.00000000000000000,0E+1,1.00000000000000000,1.00000000000000000,...,0E+1,0E+1,0E+1,1.00000000000000000,1.00000000000000000,1.00000000000000000,1.00000000000000000,1.00000000000000000,1.00000000000000000,depositTKN
3,100000,1.00000000000000000,1.00000000000000000,0E+1,1.00000000000000000,0E+1,1.00000000000000000,0E+1,1.00000000000000000,1.00000000000000000,...,0E+1,0E+1,1.00000000000000000,1.00000000000000000,1.00000000000000000,1.00000000000000000,1.00000000000000000,1.00000000000000000,1.00000000000000000,depositBNT
4,2000,1.00000000000000000,1.00000000000000000,0E+1,1.00000000000000000,0E+1,1.00000000000000006546875339062500428615744854...,0E+1,1.00000000000000000,1.00000000000000004733331360207766804141515990...,...,0E+1,0E+1,1.00000000000000000,1.00000000000000000,1.00000000000000003549407242061409204000222389...,1.00000000000000000,1.00000000000000003549407242061409204000222389...,1.00000000000000000,1.00000000000000011347079558052648603346372236...,tradeTKN
5,5332,0.99999999999999986659346801421386362123153612...,1.00000000000000000,0E+1,1.00000000000000004270287235875375824821853678...,0E+1,1.00000000000000013092659077674400611169091759...,0E+1,1.00000000000000000,1.00000000000000004727029182744758550488326373...,...,0E+1,0E+1,1.00000000000000000,1.00000000000000000,1.00000000000000003549407242061409204000222389...,1.00000000000000002201804223556105356398968909...,1.00000000000000003549407242061409204000222389...,1.00000000000000011039438897887299304035021614...,1.00000000000000007049069319862961503688136440...,tradeBNT
6,43560,0.99999999999999986659346801421386362123153612...,1.00000000000000000,0E+1,1.00000000000000004270287235875375824821853678...,0E+1,1.00000000000000013092659077674400611169091759...,0E+1,1.00000000000000000,1.00000000000000004727029182744758550488326373...,...,0E+1,0E+1,0.99999999999999996600260871022780097413099369...,0.99999999623549803825586378415574728710192377...,1.00000000000000003549407242061409204000222389...,1.00000000000000002201804223556105356398968909...,1.00000000000000003549407242061409204000222389...,1.00000000000000011039438897887299304035021614...,1.00000000000000007049069319862961503688136440...,depositBNT
7,220800,0.99999999999999986659346801421386362123153612...,1.00000000000000000,0E+1,1.00000000000000004240073384694546316552573186...,0E+1,1.00000000000000013092659077674400611169091759...,0E+1,1.00000000000000000,0.99999999999999987159893380545568246157407824...,...,0E+1,0E+1,0.99999999999999996600260871022780097413099369...,0.99999999815323937020333999118459741033793195...,0.99999999999999986561185824503331088138498031...,1.00000000000000002186232621454554672372367012...,0.99999999999999986561185824503331088138498031...,1.00000000000000005755834917512740320558901265...,1.00000000000000007049069307377519865718730733...,depositTKN
8,213,1.00000000000000010344620023017024397308179258...,1.00000000000000000,0E+1,1.00000000000000005558050324762754553020948043...,0E+1,1.00000000000000013637345941530941952932265031...,0E+1,1.00000000000000000,0.99999999999999987160234841012634790042893225...,...,0E+1,0E+1

In [27]:
result_df.to_csv('result_df.csv')

In [28]:
# comparison = simulation.compare(validation)
# comparison
result_df.columns



Index(['amount', 'tkn_dayTrader', 'tkn_tknProvider', 'tkn_bntProvider',
       'tkn_masterVault', 'tkn_epVault', 'bnt_dayTrader', 'bnt_tknProvider',
       'bnt_bntProvider', 'bnt_masterVault', 'bntkn_dayTrader',
       'bntkn_tknProvider', 'bntkn_bntProvider', 'bnbnt_dayTrader',
       'bnbnt_tknProvider', 'bnbnt_bntProvider', 'bnbnt_bntPool',
       'bntCurrentPoolFunding', 'tknStakedBalance', 'bntStakedBalance',
       'tknTradingLiquidity', 'bntTradingLiquidity'],
      dtype='object')

In [29]:
# comparison = comparison.reset_index()
# comparison

In [30]:
# import seaborn as sns
# import matplotlib.pyplot as plt
#
# for col in result_df.columns:
#     try:
#         fig = plt.figure()
#         fig.set_size_inches(6,6)
#         print('\n\n')
#         print(f"Compare {col}:")
#         sns.lineplot(data = result_df, x = 'index', y = col, color='r')
#         plt.show()
#     except:
#         pass